In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import torch 

seed = 2023
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
import dolfin as df
import time
from utils import *
from utils_training import *
import prepare_data
from utils_compare_methods import *
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd
import gc

sns.set_theme()
sns.set_context("paper")
colors = sns.color_palette("mako").as_hex()
my_cmap = sns.color_palette("viridis", as_cmap=True)

%load_ext autoreload
%autoreload 2


In [ ]:
params_array = np.load("../data/agentParams.npy")
phi_fem_sols_array = np.load("../data/W.npy")
phi_array = np.load("../data/Phi.npy")
G_array = np.load("../data/G.npy")
F_array = np.load("../data/F.npy")

In [ ]:
loss_function = Loss()
domain, domain_1, domain_2 = loss_function.compute_boundaries(
    torch.tensor(phi_array), 2
)
derivator_level2 = Derivator_fd(
    axes=[1, 2],
    interval_lengths=[1, 1],
    derivative_symbols=[(0,), (1,), (0, 0), (1, 1), (0, 1)],
)  # {"x":(0,),"y":(1,),"xx":(0,0),"yy":(1,1),"xy":(0,1)}

U = phi_array * phi_fem_sols_array + G_array
(U_x, U_y, U_xx, U_yy, U_xy) = derivator_level2(torch.tensor(U))
delta_U_on_domain = (-(U_xx + U_yy) * domain_2).numpy()

In [ ]:
maxs_delta_u_on_domain = np.max(delta_U_on_domain, axis=(1, 2))
print(max(maxs_delta_u_on_domain))
print(min(maxs_delta_u_on_domain))
print(np.mean(maxs_delta_u_on_domain))
print(np.median(maxs_delta_u_on_domain))
print(np.std(maxs_delta_u_on_domain))
print(np.argmax(maxs_delta_u_on_domain))

index = np.argmax(maxs_delta_u_on_domain)
plt.figure(figsize=(16, 12))
plt.subplot(2, 3, 1)
plt.imshow(F_array[index], origin="lower", cmap="viridis")
plt.title("F")
plt.colorbar(shrink=0.6)
plt.subplot(2, 3, 2)
plt.imshow(phi_array[index], origin="lower", cmap="viridis")
plt.colorbar(shrink=0.6)
plt.title("phi")
plt.subplot(2, 3, 3)
plt.imshow(G_array[index], origin="lower", cmap="viridis")
plt.title("G")
plt.colorbar(shrink=0.6)
plt.subplot(2, 3, 4)
plt.imshow(
    phi_fem_sols_array[index] * (phi_array[index] <= 3e-16),
    origin="lower",
    cmap="viridis",
)
plt.colorbar(shrink=0.6)
plt.title("W")
plt.subplot(2, 3, 5)
plt.imshow(phi_array[index] <= 3e-16, origin="lower", cmap="viridis")
plt.colorbar(shrink=0.6)
plt.title("domain")
plt.subplot(2, 3, 6)
plt.imshow(
    (phi_array[index] * phi_fem_sols_array[index] + G_array[index])
    * (phi_array[index] <= 3e-16),
    origin="lower",
    cmap="viridis",
)
plt.title("u")
plt.colorbar(shrink=0.6)
plt.show()

print(np.argmin(maxs_delta_u_on_domain))
index = np.argmin(maxs_delta_u_on_domain)
plt.figure(figsize=(16, 12))
plt.subplot(2, 3, 1)
plt.imshow(F_array[index], origin="lower", cmap="viridis")
plt.title("F")
plt.colorbar(shrink=0.6)
plt.subplot(2, 3, 2)
plt.imshow(phi_array[index], origin="lower", cmap="viridis")
plt.colorbar(shrink=0.6)
plt.title("phi")
plt.subplot(2, 3, 3)
plt.imshow(G_array[index], origin="lower", cmap="viridis")
plt.title("G")
plt.colorbar(shrink=0.6)
plt.subplot(2, 3, 4)
plt.imshow(
    phi_fem_sols_array[index] * (phi_array[index] <= 3e-16),
    origin="lower",
    cmap="viridis",
)
plt.colorbar(shrink=0.6)
plt.title("W")
plt.subplot(2, 3, 5)
plt.imshow(phi_array[index] <= 3e-16, origin="lower", cmap="viridis")
plt.colorbar(shrink=0.6)
plt.title("domain")
plt.subplot(2, 3, 6)
plt.imshow(
    (phi_array[index] * phi_fem_sols_array[index] + G_array[index])
    * (phi_array[index] <= 3e-16),
    origin="lower",
    cmap="viridis",
)
plt.title("u")
plt.colorbar(shrink=0.6)
plt.show()

In [ ]:
to_keep = np.where((maxs_delta_u_on_domain <= 25) & (maxs_delta_u_on_domain > 24.8))[0]
print(len(to_keep))

In [ ]:
params_array = np.load("../data_constrain_on_F_3000/agentParams.npy")
phi_fem_sols_array = np.load("../data_constrain_on_F_3000/W.npy")
phi_array = np.load("../data_constrain_on_F_3000/Phi.npy")
G_array = np.load("../data_constrain_on_F_3000/G.npy")
F_array = np.load("../data_constrain_on_F_3000/F.npy")

F_new = F_array[to_keep]
G_new = G_array[to_keep]
Phi_new = phi_array[to_keep]
params_new = params_array[to_keep]
W_new = phi_fem_sols_array[to_keep]

In [ ]:
if not (os.path.exists(f"../data_reduced_F")):
    os.makedirs(f"../data_reduced_F")


print("Save F,G,agentParams,W, nb_Data=", len(F_new))
np.save(f"../data_reduced_F/F.npy", F_new)
np.save(f"../data_reduced_F/Phi.npy", Phi_new)
np.save(
    f"../data_reduced_F/agentParams.npy",
    params_new,
)
np.save(f"../data_reduced_F/W.npy", W_new)
np.save(f"../data_reduced_F/G.npy", G_new)